In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
import networkx as nx

import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

from shapely.geometry import Point
from shapely.geometry import Polygon

import glob
import os
import itertools
import tqdm
import gc
import time
import pickle

from joblib import Parallel, delayed

import configparser
cfg = configparser.ConfigParser()
cfg.optionxform = str
cfg.read('/home/sarth/rootdir/assets/global.ini')
cfg = {s: dict(cfg.items(s)) for s in cfg.sections()}
PATHS = cfg['PATHS']

In [2]:
DIRNAME = '03min_GloFAS_CAMELS-IND'
SAVE_PATH = os.path.join(PATHS['devp_datasets'], DIRNAME)
os.makedirs(SAVE_PATH, exist_ok=True)
resolution = 0.05

In [8]:
camels_attributes = pd.read_csv(os.path.join(PATHS['CAMELS'], 'CAMELS-IND', 'CAMELS_IND_Catchments_Streamflow_Sufficient', 'attributes_csv', 'camels_ind_topo.csv'))

temp = pd.read_csv(os.path.join(PATHS['CAMELS'], 'CAMELS-IND', 'CAMELS_IND_Catchments_Streamflow_Sufficient', 'attributes_csv', 'camels_ind_name.csv'))
temp = temp[['gauge_id', 'cwc_site_name', 'ghi_stn_id', 'river_basin', 'cwc_river', 'flow_availability']]
camels_attributes = camels_attributes.merge(temp, on='gauge_id', how='left')

camels_attributes['gauge_id'] = camels_attributes['gauge_id'].map(lambda x: str(x).zfill(5))
camels_attributes['huc_02'] = camels_attributes['gauge_id'].map(lambda x: x[:2])
camels_attributes = camels_attributes[['gauge_id', 'huc_02', 'ghi_lon', 'ghi_lat', 'ghi_area', 'cwc_lon', 'cwc_lat', 'cwc_area', 'cwc_site_name', 'ghi_stn_id', 'river_basin', 'cwc_river', 'flow_availability']]
camels_attributes = camels_attributes.set_index('gauge_id')
camels_attributes.sort_values(by = 'ghi_area', ascending = True, inplace = True)
camels_attributes

,huc_02,ghi_lon,ghi_lat,ghi_area,cwc_lon,cwc_lat,cwc_area,cwc_site_name,ghi_stn_id,river_basin,cwc_river,flow_availability
gauge_id,,,,,,,,,,,,
14015,14,73.11033,18.73540,125.7,73.1108,18.7367,125.0,Pen,wfrn_penxx,WFRN,Bhogeswari,31.36
14009,14,73.28122,18.23122,292.3,73.2833,18.2317,259.0,Mangaon (Seasonal),wfrn_manga,WFRN,Savitri/Kal,67.40
15006,15,74.88124,13.51876,299.6,74.8800,13.5214,253.0,Avershe,wfrs_avers,WFRS,Seetha,39.00
05025,05,78.05617,11.93959,356.0,78.0572,11.9383,362.0,Thoppur,cauv_thopp,Cauvery,Cauvery/Thoppaiyar,31.52
15032,15,74.98123,13.29791,356.9,74.9806,13.2942,327.0,Yennehole,wfrs_yenne,WFRS,Swarna,70.67
...,...,...,...,...,...,...,...,...,...,...,...,...
04062,04,80.06876,16.79375,240055.4,80.0692,16.7942,235544.0,Wadenapally,kris_waden,Krishna,Krishna,95.43
04060,04,80.61874,16.49796,257260.0,80.6250,16.5011,251360.0,Vijayawada,kris_vijay,Krishna,Krishna,84.90
03073,03,80.39375,18.58542,267340.4,80.3958,18.5872,268200.0,Perur,goda_perur,Godavari,Godavari,94.32


In [4]:
uparea = xr.open_dataset(os.path.join(PATHS['gis_ldd'], 'GloFAS_03min/upstream_area_km2.nc'))
ds_varname = list(uparea.data_vars)[0]
uparea = uparea[ds_varname]
uparea.load()

<xarray.DataArray 'upstream_area_km2' (lat: 3000, lon: 7200)> Size: 86MB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * lon      (lon) float64 58kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * lat      (lat) float64 24kB 89.97 89.92 89.88 89.82 ... -59.88 -59.92 -59.97

In [5]:
ldd = xr.open_dataset(os.path.join(PATHS['gis_ldd'], 'GloFAS_03min', 'ldd.nc'))
ds_var_name = list(ldd.data_vars)[0]
ldd = ldd[ds_var_name]
ldd.load()

<xarray.DataArray 'ldd' (lat: 3000, lon: 7200)> Size: 86MB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * lon      (lon) float64 58kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * lat      (lat) float64 24kB 89.97 89.92 89.88 89.82 ... -59.88 -59.92 -59.97
Attributes:
    long_name:     Local drain direction
    grid_mapping:  crs

In [6]:
def get_contributing_region(ldd, lon_snapped, lat_snapped):
    # ldd_loc = lambda x,y: int(ldd.sel(lat = y, lon = x, method = 'nearest').values)
    def ldd_loc(x, y):
        value = ldd.sel(lon=x, lat=y, method = 'nearest').values
        if np.isnan(value):
            return 5
        else:
            return int(value)
    # print(lon_snapped, lat_snapped, ldd_loc(lon_snapped, lat_snapped))

    def get_nbr_list(x, y, res = resolution):
        # get the 8 neighbors
        nbr = [
            (x-res, y-res), (x, y-res), (x+res, y-res), 
            (x-res, y), (x+res, y),
            (x-res, y+res), (x, y+res), (x+res, y+res)]
        # Round to 3
        nbr = [(round(x, 3), round(y, 3)) for x, y in nbr]
        return nbr
    
    def get_nbr_ldd(nbr):
        return [ldd_loc(x, y) for x, y in nbr]
    
    def get_nbr_contributing(nbr, ldd_nbr):
        # Sequence: 9, 8, 7, 6, 4, 3, 2, 1
        seq_contributing = [9, 8, 7, 6, 4, 3, 2, 1]
        nbr_contributing = [nbr[i] if ldd_nbr[i] == seq_contributing[i] else None for i in range(8)]
        nbr_contributing = [x for x in nbr_contributing if x is not None]
        return nbr_contributing

    loc_to_visit = [(lon_snapped, lat_snapped)]
    loc_visited = []

    while len(loc_to_visit) > 0:
        loc = loc_to_visit.pop(0)
        loc_visited.append(loc)
        x, y = loc
        nbr = get_nbr_list(x, y)
        ldd_nbr = get_nbr_ldd(nbr)
        nbr_contributing = get_nbr_contributing(nbr, ldd_nbr)
        for nbr in nbr_contributing:
            if nbr not in loc_visited and nbr not in loc_to_visit:
                loc_to_visit.append(nbr)
        # print(len(loc_to_visit), len(loc_visited))
    
    # print(f"Number of contributing cells: {len(loc_visited)}")

    nodes_coords = pd.DataFrame(loc_visited, columns = ['lon', 'lat'])

    # fig, ax = plt.subplots(1,1,figsize = (10, 10))
    # ax.scatter(nodes_coords['lon'], nodes_coords['lat'], s = 1, c = 'blue', label = 'Contributing Cells')
    # ax.scatter(lon_snapped, lat_snapped, c = 'red', s = 10, label = 'Outlet Location')
    # ax.legend()
    # ax.set_aspect('equal')
    # ax.grid()
    # plt.show()

    def get_edge(loc, res = resolution):
        lon, lat = loc
        ldd_val = ldd_loc(lon, lat)
        if ldd_val == 1: # South-West
            edge_loc = (lon-res, lat-res)
        elif ldd_val == 2: # South
            edge_loc = (lon, lat-res)
        elif ldd_val == 3: # South-East
            edge_loc = (lon+res, lat-res)
        elif ldd_val == 4: # West
            edge_loc = (lon-res, lat)
        elif ldd_val == 6: # East
            edge_loc = (lon+res, lat)
        elif ldd_val == 7: # North-West
            edge_loc = (lon-res, lat+res)
        elif ldd_val == 8: # North
            edge_loc = (lon, lat+res)
        elif ldd_val == 9: # North-East
            edge_loc = (lon+res, lat+res)
        elif ldd_val == 5: # Pit
            edge_loc = None
        if edge_loc is not None:
            edge_loc = (round(edge_loc[0], 3), round(edge_loc[1], 3))
        return edge_loc

    edges = [(loc, get_edge(loc)) for loc in loc_visited[::-1][:-1]]
    # print(len(edges))

    # edges: ((lon_1, lat_1), (lon_2, lat_2)) -> edge_list: (idx_1, idx_2)
    def replace_loc_by_idx(loc, nodes_coords):
        return nodes_coords[(nodes_coords['lon'] == loc[0]) & (nodes_coords['lat'] == loc[1])].index[0]
    
    edges_idx = [(replace_loc_by_idx(edge[0], nodes_coords), replace_loc_by_idx(edge[1], nodes_coords)) for edge in edges if edge[1] is not None]

    
    # os.makedirs(os.path.join(SAVE_PATH, 'graph_files'), exist_ok = True)
    # os.makedirs(os.path.join(SAVE_PATH, 'graph_files', huc), exist_ok = True)
    # os.makedirs(os.path.join(SAVE_PATH, 'graph_files', huc, gauge_id), exist_ok = True)
    # SAVEPATH = os.path.join(SAVE_PATH, 'graph_files', huc, gauge_id)

    # nodes_coords.to_csv(os.path.join(SAVEPATH, 'nodes_coords.csv'), index = True)

    edges = pd.DataFrame(edges, columns = ['from', 'to'])
    edges_idx = pd.DataFrame(edges_idx, columns = ['from', 'to'])
    edges['from_idx'] = edges_idx['from']
    edges['to_idx'] = edges_idx['to']
    # edges.to_csv(os.path.join(SAVEPATH, 'edges.csv'), index = True)

    # # Return number of nodes and edges
    # num_nodes = len(nodes_coords)
    # num_edges = len(edges)
    # return num_nodes, num_edges

    return nodes_coords, edges

In [7]:
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union

def snap_on_grid(
    grid_uparea: xr.DataArray,
    gauge_lon: float,
    gauge_lat: float,
    gauge_uparea: float,
    resolution: float,
    grid_ldd: xr.DataArray,
    huc: str,
    gauge_id: str,
    plot_title = ''
    ):
    assert 'lat' in grid_uparea.coords, 'latitude not in grid_uparea.coords'
    assert 'lon' in grid_uparea.coords, 'longitude not in grid_uparea.coords'
    is_lat_increasing = grid_uparea.lat[1].values > grid_uparea.lat[0].values
    is_lon_increasing = grid_uparea.lon[1].values > grid_uparea.lon[0].values
    if not is_lat_increasing:
        grid_uparea = grid_uparea.sortby('lat')
    if not is_lon_increasing:
        grid_uparea = grid_uparea.sortby('lon')

    # Ensure that both lat and lon are round(x, 3)
    grid_uparea['lat'] = grid_uparea['lat'].round(3)
    grid_uparea['lon'] = grid_uparea['lon'].round(3)

    grid_ldd['lat'] = grid_ldd['lat'].round(3)
    grid_ldd['lon'] = grid_ldd['lon'].round(3)
    
    # Find nearest grid cell
    grid_nearest = grid_uparea.sel(
        lat = gauge_lat,
        lon = gauge_lon,
        method = 'nearest'
    )
    nearest_lat = float(grid_nearest.lat.values)
    nearest_lon = float(grid_nearest.lon.values)

    grid_uparea = grid_uparea.sel(
        lat = slice(nearest_lat - 2*resolution, nearest_lat + 2*resolution),
        lon = slice(nearest_lon - 2*resolution, nearest_lon + 2*resolution)
    )
    
    # fig, ax = plt.subplots()
    # grid_uparea.plot(ax = ax, cmap = 'Blues', edgecolor = 'k')
    # ax.plot(gauge_lon, gauge_lat, 'ro')
    # ax.plot(nearest_lon, nearest_lat, 'bo')
    # plt.show()

    # Create DataArray UPA (upstream area accordance) = cell_uparea/gauge_uparea if gauge_uparea > cell_uparea else gauge_uparea/cell_uparea
    UPA = grid_uparea.copy()
    UPA = xr.where(UPA > gauge_uparea, gauge_uparea / UPA, UPA / gauge_uparea)
    
    camels = gpd.read_file(os.path.join(PATHS['CAMELS'], 'CAMELS-IND', 'CAMELS_IND_Catchments_Streamflow_Sufficient', 'shapefiles_catchment', 'catchments.shp'), crs='EPSG:4326')
    camels = camels[camels['gauge_id'] == gauge_id]

    def calculate_iou(lon_under_consideration, lat_under_consideration):
        nodes_coords, edges = get_contributing_region(grid_ldd, lon_under_consideration, lat_under_consideration)
        edge_list = edges.loc[:,['from_idx', 'to_idx']].values.tolist()

        # fig, ax = plt.subplots(1,1,figsize = (10, 10))
        # camels.plot(ax = ax, color = 'none', edgecolor = 'black', lw = 0.5)
        # for idx, row in nodes_coords.iterrows():
        #     x, y = row[['lon', 'lat']]
        #     ax.plot(x, y, 'o', color = 'gray', markersize = 2)
        #     ax.add_patch(plt.Rectangle((x-resolution/2, y-resolution/2), resolution, resolution, fill=None, edgecolor='cornflowerblue', lw=0.5))
        # ax.plot(nearest_lon, nearest_lat, 'o', color = 'red', markersize = 4)
        # ax.plot(gauge_lon, gauge_lat, '^', color = 'red', markersize = 4)
        # for edge in edge_list:
        #     from_idx, to_idx = edge
        #     from_x, from_y = nodes_coords.loc[from_idx, ['lon', 'lat']]
        #     to_x, to_y = nodes_coords.loc[to_idx, ['lon', 'lat']]
        #     ax.annotate("", xy=(to_x, to_y), xytext=(from_x, from_y), arrowprops=dict(arrowstyle="-|>", mutation_scale=7, color = 'cornflowerblue', lw = 0.5))
        # ax.set_aspect('equal')
        # plt.show()

        # IOU = area of intersection / area of union
        # Create low resolution polygon from contributing nodes
        contributing_cells = []
        for idx, row in nodes_coords.iterrows():
            x, y = row[['lon', 'lat']]
            cell = Polygon([
                (x - resolution/2, y - resolution/2),
                (x - resolution/2, y + resolution/2),
                (x + resolution/2, y + resolution/2),
                (x + resolution/2, y - resolution/2)
            ])
            contributing_cells.append(cell)
        low_res_polygon = unary_union(contributing_cells)

        # Get high-resolution polygon from CAMELS
        high_res_polygon = camels.geometry.iloc[0]

        # Calculate intersection and union areas
        intersection_area = low_res_polygon.intersection(high_res_polygon).area
        union_area = low_res_polygon.union(high_res_polygon).area

        # Calculate IoU
        iou = intersection_area / union_area

        # Update the plot to include the low_res_polygon and high_res_polygon
        # fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        # gpd.GeoSeries([high_res_polygon]).plot(ax=ax, color='none', edgecolor='black', lw=0.5)
        # gpd.GeoSeries([low_res_polygon]).plot(ax=ax, color='none', edgecolor='red', lw=1)
        # for idx, row in nodes_coords.iterrows():
        #     x, y = row[['lon', 'lat']]
        #     ax.plot(x, y, 'o', color='gray', markersize=2)
        # ax.plot(nearest_lon, nearest_lat, 'o', color='red', markersize=4)
        # ax.plot(gauge_lon, gauge_lat, '^', color='red', markersize=4)
        # for edge in edge_list:
        #     from_idx, to_idx = edge
        #     from_x, from_y = nodes_coords.loc[from_idx, ['lon', 'lat']]
        #     to_x, to_y = nodes_coords.loc[to_idx, ['lon', 'lat']]
        #     ax.annotate("", xy=(to_x, to_y), xytext=(from_x, from_y), arrowprops=dict(arrowstyle="-|>", mutation_scale=7, color='cornflowerblue', lw=0.5))
        # ax.set_aspect('equal')
        # plt.title(f"IoU: {iou:.4f}")
        # plt.show()

        # print(f"Intersection area: {intersection_area:.4f}")
        # print(f"Union area: {union_area:.4f}")
        # print(f"IoU: {iou:.4f}")  

        return iou

    # Calculate IoU for all grid cells
    iou = UPA.copy()
    for lon_val in UPA.lon.values:
        for lat_val in UPA.lat.values:
            # Only calculate where grid_uparea is not nan
            if not np.isnan(grid_uparea.sel(lon = lon_val, lat = lat_val).values):
                iou_val = calculate_iou(lon_val, lat_val)
                iou.loc[dict(lon = lon_val, lat = lat_val)] = iou_val
            else:
                iou.loc[dict(lon = lon_val, lat = lat_val)] = np.nan
            # iou.loc[dict(lon = lon_val, lat = lat_val)] = calculate_iou(lon_val, lat_val)  

    # ED (Euclidian distance) = \sqrt{ (1 - UPA)^2 + (1 - iou)^2 }
    ED = np.sqrt((1 - UPA)**2 + (1 - iou)**2)
    # Plot ED
    # fig, ax = plt.subplots()
    # ED.plot(ax = ax, cmap = 'Blues', edgecolor = 'k')
    # ax.plot(nearest_lon, nearest_lat, 'ro')
    # ax.plot(gauge_lon, gauge_lat, 'r^')
    # plt.show()

    # Dataframe of ED with columns lon, lat and ED
    ED_df = ED.to_dataframe().reset_index()
    ED_df = ED_df.rename(columns = {ds_varname: 'ED'})
    ED_df['huc'] = huc
    ED_df['gauge_id'] = gauge_id
    # # Distance to gauge location
    ED_df['distance'] = np.sqrt((ED_df['lon'] - gauge_lon)**2 + (ED_df['lat'] - gauge_lat)**2)
    # Sort by ED and then by distance
    ED_df = ED_df.sort_values(by = ['ED', 'distance'], ascending = [True, True])

    # Select first cell as snapped location.
    lon_snapped = ED_df['lon'].iloc[0]
    lat_snapped = ED_df['lat'].iloc[0]
    # Get nodes and edges
    nodes_coords, edges = get_contributing_region(grid_ldd, lon_snapped, lat_snapped)
    # Save nodes and edges
    SAVEPATH = os.path.join(SAVE_PATH, 'graph_files', huc, gauge_id)
    os.makedirs(SAVEPATH, exist_ok = True)
    nodes_coords.to_csv(os.path.join(SAVEPATH, 'nodes_coords.csv'), index = True)
    edges.to_csv(os.path.join(SAVEPATH, 'edges.csv'), index = True)
    # uparea at snapped_location
    uparea_snapped = grid_uparea.sel(lon = lon_snapped, lat = lat_snapped).values
    # iou at snapped_location
    iou_snapped = calculate_iou(lon_snapped, lat_snapped)
    # area_percent_difference
    area_percent_difference = (abs(gauge_uparea - uparea_snapped) / gauge_uparea) * 100

    num_nodes = len(nodes_coords)
    num_edges = len(edges)
    edge_list = edges.loc[:,['from_idx', 'to_idx']].values.tolist()

    fig, ax = plt.subplots(1,1,figsize = (15, 15))
    camels.plot(ax = ax, color = 'none', edgecolor = 'black', lw = 0.5)
    for idx, row in nodes_coords.iterrows():
        x, y = row[['lon', 'lat']]
        ax.plot(x, y, 'o', color = 'gray', markersize = 2)
        ax.add_patch(plt.Rectangle((x-resolution/2, y-resolution/2), resolution, resolution, fill=None, edgecolor='cornflowerblue', lw=0.5))
    ax.plot(nearest_lon, nearest_lat, 'o', color = 'red', markersize = 4)
    ax.plot(gauge_lon, gauge_lat, '^', color = 'red', markersize = 4)
    ax.plot(lon_snapped, lat_snapped, 'o', color = 'green', markersize = 4)
    for edge in edge_list:
        from_idx, to_idx = edge
        from_x, from_y = nodes_coords.loc[from_idx, ['lon', 'lat']]
        to_x, to_y = nodes_coords.loc[to_idx, ['lon', 'lat']]
        ax.annotate("", xy=(to_x, to_y), xytext=(from_x, from_y), arrowprops=dict(arrowstyle="-|>", mutation_scale=7, color = 'cornflowerblue', lw = 0.5))
    ax.set_aspect('equal')
    plt.title(plot_title)
    # plt.show()
    os.makedirs(os.path.join(SAVE_PATH, 'graph_plots', huc), exist_ok = True)
    plt.savefig(os.path.join(SAVE_PATH, 'graph_plots', huc, f"{gauge_id}.pdf"))
    plt.close()

    return lon_snapped, lat_snapped, uparea_snapped, iou_snapped, area_percent_difference, num_nodes, num_edges

# row = camels_attributes.iloc[0]
# temp = snap_on_grid(uparea, row['gauge_lon'], row['gauge_lat'], row['area_geospa_fabric'], resolution, ldd, row['huc_02'], row.name)

In [10]:
row = camels_attributes.iloc[0]
huc_id, gauge_lon, gauge_lat, gauge_uparea = row['huc_02'], row['ghi_lon'], row['ghi_lat'], row['ghi_area']
plot_title = f"Outlet: {row.name} | Station: {row['cwc_site_name']} | River: {row['cwc_river']} | River Basin: {row['river_basin']} | Flow Availability: {row['flow_availability']}"
lon_snapped, lat_snapped, uparea_snapped, iou_snapped, area_percent_difference, num_nodes, num_edges = snap_on_grid(
    uparea,
    gauge_lon,
    gauge_lat,
    gauge_uparea,
    resolution,
    ldd,
    huc_id,
    gauge_id = row.name,
    plot_title = plot_title
)

In [11]:
camels_attributes_snapped = camels_attributes.copy()
for gauge_id, gauge_info in tqdm.tqdm(camels_attributes.iterrows(), total = len(camels_attributes)):
    # huc_id, gauge_lon, gauge_lat, gauge_uparea = gauge_info.values
    huc_id, gauge_lon, gauge_lat, gauge_uparea = gauge_info['huc_02'], gauge_info['ghi_lon'], gauge_info['ghi_lat'], gauge_info['ghi_area']
    plot_title = f"Outlet: {gauge_id} | Station: {gauge_info['cwc_site_name']} | River: {gauge_info['cwc_river']} | River Basin: {gauge_info['river_basin']} | Flow Availability: {gauge_info['flow_availability']}"
    try:
        lon_snapped, lat_snapped, uparea_snapped, iou_snapped, area_percent_difference, num_nodes, num_edges = snap_on_grid(
            uparea,
            gauge_lon,
            gauge_lat,
            gauge_uparea,
            resolution,
            ldd,
            huc_id,
            gauge_id,
            plot_title = plot_title
        )
    except:
        lon_snapped, lat_snapped, uparea_snapped, iou_snapped, area_percent_difference, num_nodes, num_edges = None, None, None, None, None, None, None
    camels_attributes_snapped.loc[gauge_id, 'snapped_lon'] = lon_snapped
    camels_attributes_snapped.loc[gauge_id, 'snapped_lat'] = lat_snapped
    camels_attributes_snapped.loc[gauge_id, 'snapped_uparea'] = uparea_snapped
    camels_attributes_snapped.loc[gauge_id, 'snapped_iou'] = iou_snapped
    camels_attributes_snapped.loc[gauge_id, 'area_percent_difference'] = area_percent_difference
    camels_attributes_snapped.loc[gauge_id, 'num_nodes'] = num_nodes
    camels_attributes_snapped.loc[gauge_id, 'num_edges'] = num_edges
camels_attributes_snapped.head()

100%|██████████| 242/242 [1:52:28<00:00, 27.89s/it] 


,huc_02,ghi_lon,ghi_lat,ghi_area,cwc_lon,cwc_lat,cwc_area,cwc_site_name,ghi_stn_id,river_basin,cwc_river,flow_availability,snapped_lon,snapped_lat,snapped_uparea,snapped_iou,area_percent_difference,num_nodes,num_edges
gauge_id,,,,,,,,,,,,,,,,,,,
14015,14,73.11033,18.73540,125.7,73.1108,18.7367,125.0,Pen,wfrn_penxx,WFRN,Bhogeswari,31.36,73.125,18.725,116.744995,0.650870,7.124106,4.0,3.0
14009,14,73.28122,18.23122,292.3,73.2833,18.2317,259.0,Mangaon (Seasonal),wfrn_manga,WFRN,Savitri/Kal,67.40,73.325,18.225,380.302094,0.520864,30.106777,13.0,12.0
15006,15,74.88124,13.51876,299.6,74.8800,13.5214,253.0,Avershe,wfrs_avers,WFRS,Seetha,39.00,74.925,13.475,329.444855,0.607418,9.961565,11.0,10.0
05025,05,78.05617,11.93959,356.0,78.0572,11.9383,362.0,Thoppur,cauv_thopp,Cauvery,Cauvery/Thoppaiyar,31.52,78.125,11.975,331.356476,0.634196,6.922338,11.0,10.0
15032,15,74.98123,13.29791,356.9,74.9806,13.2942,327.0,Yennehole,wfrs_yenne,WFRS,Swarna,70.67,74.975,13.275,329.717102,0.755086,7.616389,11.0,10.0


In [12]:
camels_attributes_snapped.to_csv(os.path.join(SAVE_PATH, 'graph_attributes.csv'), index = True)

In [13]:
camels_graph = camels_attributes_snapped.copy()
print(camels_graph.shape)
camels_graph = camels_graph[camels_graph['ghi_area'] <= 30000]
print(camels_graph.shape)
camels_graph = camels_graph[camels_graph['area_percent_difference'] < 10]
print(camels_graph.shape)
camels_graph = camels_graph[camels_graph['num_nodes'] > 1]
print(camels_graph.shape)

(242, 19)
(200, 19)
(191, 19)
(191, 19)


In [14]:
camels_graph.sort_values(ascending=True, by = 'huc_02').groupby('huc_02').size()

huc_02
03    37
04    31
05    16
06     4
07     7
08    16
09     5
10     5
11     5
12    13
13     6
14     8
15    21
16     5
17    12
dtype: int64